A new module, Sfilter.kinetics, was introduced in version 0.6.1 to replace the Sfilter.MSM.
There are several reasons for this rewrite:  
A. We acknowledge that the permeation mechanism is non-Markovian.  
B. The old version used too much memory, and in this version, performance was taken into consideration from the beginning.  

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import Sfilter
from pathlib import Path
import sys
from pympler.asizeof import asizeof

In [2]:
base = Path("../")
f_list = [base/f"{i:02d}/analysis/08-02-1.8A/k_cylinder.log" for i in range(10)]

In [3]:
%%time
k_model = Sfilter.kinetics.Sf_model(f_list, traj_dtype=np.int64)
msm = Sfilter.MSM.SF_msm(f_list)
msm.calc_state_array()


CPU times: user 3min 15s, sys: 3.55 s, total: 3min 18s
Wall time: 3min 21s


# np.int64

In [ ]:
%%time
k_model = Sfilter.kinetics.Sf_model(f_list, traj_dtype=np.int64)
%%time
_p = k_model.calc_passage_time()
print("Size of the new model       (MB):", asizeof(k_model) / 1e6)

In [5]:
%%time
_p = k_model.calc_passage_time()

CPU times: user 47 s, sys: 36 ms, total: 47.1 s
Wall time: 47.1 s


# np.int32

In [7]:
%%time
k_model = Sfilter.kinetics.Sf_model(f_list, traj_dtype=np.int32)

CPU times: user 2min 51s, sys: 1 s, total: 2min 52s
Wall time: 2min 52s


In [8]:
%%time
_p = k_model.calc_passage_time()

CPU times: user 1min 14s, sys: 51.9 ms, total: 1min 14s
Wall time: 1min 14s


# np.int8

In [12]:
%%time
k_model = Sfilter.kinetics.Sf_model(f_list)

CPU times: user 2min 48s, sys: 1.04 s, total: 2min 49s
Wall time: 2min 49s


In [13]:
%%time
_p = k_model.calc_passage_time()

CPU times: user 1min 13s, sys: 28.1 ms, total: 1min 13s
Wall time: 1min 13s


In [14]:
print("Size of the old model       (MB):", asizeof(msm) / 1e6)
print("Size of the old traj string (MB):", asizeof(msm.state_str) / 1e6)
print("Size of the old traj int    (MB):", asizeof(msm.state_array) / 1e6)
print("Size of the new model       (MB):", asizeof(k_model) / 1e6)
print("Size of the new traj raw    (MB):", asizeof(k_model.traj_raw) / 1e6)
print("Size of the new traj lumped (MB):", asizeof(k_model.traj_node) / 1e6)


Size of the old model       (MB): 360.02468
Size of the old traj string (MB): 320.001384
Size of the old traj int    (MB): 40.0014
Size of the new model       (MB): 118.86032
Size of the new traj raw    (MB): 5.0014
Size of the new traj lumped (MB): 5.0014


In [15]:
print("Size of the passage_time_point_every_traj  (MB):", asizeof(k_model.passage_time_point_every_traj) / 1e6)
print("Size of the passage_time_length_every_traj (MB):", asizeof(k_model.passage_time_length_every_traj) / 1e6)

Size of the passage_time_point_every_traj  (MB): 53.701928
Size of the passage_time_length_every_traj (MB): 53.701928


In [17]:
mfpt_01, passage_list = msm.get_mfpt_rate_pair(0,1)
mfpt_10, passage_list = msm.get_mfpt_rate_pair(1,0)

print(mfpt_01, mfpt_10)
print(passage_list[:10])

0.14719231884751227 0.28788068984546805
[1, 4, 3, 2, 2, 5, 4, 2, 1, 1]


In [18]:
mfpt_m, mfpt_every_traj = k_model.get_mfpt()
print(1/mfpt_m[0,1], 1/mfpt_m[1,0])
print(k_model.passage_time_length_every_traj[0][1][0][:10])

0.14719231884751227 0.28788068984546805
[1 4 3 2 2 5 4 2 1 1]
